In [ ]:
# export
from pathlib import Path
from collections import defaultdict
import re
import os
import json
import logging

In [ ]:
# export
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s",
    level=logging.DEBUG,
)

In [ ]:
# export
def read_nb(fname):
    "Read the notebook in `fname`."

    data = json.loads(Path(fname).read_text())

    for c in data["cells"]:
        c["source"] = "".join(c["source"])

    return data

In [ ]:
# export

pattern: str = r"^\s*\#\s*export[s]?\s*([\w\/\.]*)?$"
pat = re.compile(pattern, re.IGNORECASE | re.MULTILINE)

In [ ]:
m = pat.search("# export")
m.group(1)

''

In [ ]:
m = pat.search(
    """# export /User/test.py
from pathlib import Path
from collections import defaultdict
import re
import os
import json
"""
)
assert m.group(1) == "/User/test.py"

In [ ]:
m = pat.search(
    """# exports
from pathlib import Path
from collections import defaultdict
import re
import os
import json
"""
)
assert m.group(1) == ""

In [ ]:
# export
def nb2script(nb_path, script_path, replace=True, cli=False, msg=True):

    nb_path = Path(nb_path)
    script_path = Path(script_path)

    if script_path.exists() and replace == False:
        raise FileExistsError(f"The file {script_path} already exists.")

    data = read_nb(nb_path)

    cells = [
        c["source"]
        for c in data["cells"]
        if c["cell_type"] == "code" and pat.search(c["source"]) is not None
    ]

    # other files where we want to write
    # for example test files
    extras = defaultdict(list)

    with open(script_path, "w") as f:
        if cli:

            f.write(f"#!/usr/bin/env python3" + "\n\n")

        if msg:

            f.write(
                f"# AUTOGENERATED! DO NOT EDIT! File to edit: {str(nb_path)} (unless otherwise specified)."
                + "\n"
            )

        for c in cells:

            # export (/some/path/file.py)
            if pat.search(c).group(1):
                extra_fname = pat.search(c).group(1)
                extras[extra_fname].append(pat.sub("", c))
                logging.info(f"Found extra name '{extra_fname}' in file")

            else:
                f.write(pat.sub("", c) + "\n\n")

    logging.info(f"Extra files to write: {repr(extras)}")
    for key, value in extras.items():
        path_to_write = Path(key)

        with open(path_to_write, "w") as f:

            if cli:

                f.write(f"#!/usr/bin/env python3" + "\n\n")

            if msg:

                f.write(
                    f"# AUTOGENERATED! DO NOT EDIT! File to edit: {str(nb_path)} (unless otherwise specified)."
                    + "\n"
                )

            for cell in value:
                f.write(pat.sub("", cell) + "\n\n")

In [ ]:
# export
def main(nb_path, script_path=None, replace=True, cli=False, msg=True):

    logging.info(f"Converting {str(nb_path)} to {str(script_path)}, replace={replace}, cli={cli}, msg={msg}")

    nb_path = Path(nb_path)
    if not script_path:
        script_path = nb_path.with_suffix(".py")

    nb2script(nb_path, script_path, replace=replace, cli=cli, msg=msg)

The following cell is for testing. It will be exported to a file in the specified path (which is .gitignore'd)

In [ ]:
# export /Users/r/Projects/Python/nb2script/test_nb.py

def f():
    return 1

In [ ]:
main("./nb2script.ipynb", "./nb2s.py")

2021-06-22 18:12:57,973 : INFO : Converting ./nb2script.ipynb to ./nb2s.py, replace=True, cli=False, msg=True
2021-06-22 18:12:57,977 : INFO : Found extra name '/Users/r/Projects/Python/nb2script/test_nb.py' in file
2021-06-22 18:12:57,979 : INFO : Extra files to write: defaultdict(<class 'list'>, {'/Users/r/Projects/Python/nb2script/test_nb.py': ['\n\ndef f():\n    return 1']})


In [ ]:
main("./nb2script.ipynb", "/Users/r/dotfiles/bin/nb2s", cli=True)

2021-06-22 18:12:58,411 : INFO : Converting ./nb2script.ipynb to /Users/r/dotfiles/bin/nb2s, replace=True, cli=True, msg=True
2021-06-22 18:12:58,415 : INFO : Found extra name '/Users/r/Projects/Python/nb2script/test_nb.py' in file
2021-06-22 18:12:58,417 : INFO : Extra files to write: defaultdict(<class 'list'>, {'/Users/r/Projects/Python/nb2script/test_nb.py': ['\n\ndef f():\n    return 1']})


In [ ]:
# export
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--replace",
        help="wether to replace the script or not",
        action="store_true",
        default=True,
    )

    parser.add_argument(
        "--cli",
        help="add shebang when the output will be used as a CLI",
        action="store_true",
        default=False,
    )

    parser.add_argument(
        "--no-msg",
        help="do not include AUTOGENERATED message in file with path to the original notebook",
        action="store_true",
        default=False,
    )

    parser.add_argument(
        "-n",
        "--nb",
        help="path to the notebook to convert",
        type=str,
        default=None,
        required=True,
    )

    parser.add_argument(
        "-s",
        "--script",
        help="path where the script will be saved",
        type=str,
        default=None,
        required=True,
    )

    args = parser.parse_args()

    msg = True

    if args.no_msg:
        msg = False

    exit(main(args.nb, args.script, args.replace, args.cli, msg))